## Final Assignment 

#### Part 1. 
This AccessViz finds the from the datafolder all the matrices that the user has specified by assigning a list of integer
values that corresponds to YKR_ID found from the MetropAccess_YKR_grid. 
     - AccessViz should inform the user about how the execution process is going. 

##### Find datafiles using os.path.join()-function. 
- This should work for all of the files in the Helsinki Travel Times folder, the user just has to pas the the YKR_IDs as a list of integers, with the seperator ';'. 
- I should also manage to tell the user how the executing process is going for the files. 

In [1]:
import geopandas as gpd
import pandas as pd
import os

#read and take a look at the YKR_grid shp file 
grid_fp = 'data/MetropAccess_YKR_grid/MetropAccess_YKR_grid_EurefFIN.shp'

#read the file
grid = gpd.read_file(grid_fp)

# Change the name of YKR_ID to from_id, to enable the join
grid.rename(columns={'YKR_ID': 'from_id'}, inplace=True)

# See the column names and confirm that we now have a column called 'from_id'
grid.head()
#grid.crs

,x,y,from_id,geometry
0,381875.0,6697880.0,5785640,"POLYGON ((382000.0001388059 6697750.000128186,..."
1,382125.0,6697880.0,5785641,"POLYGON ((382250.00013875 6697750.000128181, 3..."
2,382375.0,6697880.0,5785642,"POLYGON ((382500.0001386951 6697750.000128172,..."
3,382625.0,6697880.0,5785643,"POLYGON ((382750.0001386406 6697750.000128165,..."
4,381125.0,6697630.0,5787544,"POLYGON ((381250.000138978 6697500.000128254, ..."


In [ ]:
import glob

#YKR_IDs = [5855714, 5878087]
#print(YKR_IDs)

#use glob function to read in all the .txt-files in the data-directory
filepaths = glob.glob('data/*.txt')
#print(filepaths)

#read the filepaths in filepaths-list  into geodataframe
for idx, fp in enumerate(filepaths):
    #print(fp)
    data = pd.read_csv(fp, sep=';')
    
    '''Drop NaN values'''
    
    '''merge data (pd) and grid(gpd) together on 'from_id'  '''
    data = data.merge(grid, on='from_id')
    #print(data.head())
   
    '''Create into geodataframe'''
    #set CRS
    CRS = grid.crs
    #make in into a geodataframe
    travel_times = gpd.GeoDataFrame(data, crs=CRS, geometry ='geometry')
    #print(travel_times.head())
    
    '''Save and store each of the geodataframes into separate shapefiles called the YKR_ID of the file'''
    
    for travel_time in travel_times:
        YKR_ID = travel_times['to_id']
        travel_times.to_file(filename='YKR_ID')
        #print(YKR_ID)
    
 
    


    

In [ ]:
travel_times.head()

#### PART 2: 
Create Shapefiles from the chosen Matrix text tables (e.g. travel_times_to_5797076.txt) by joining the Matrix file with MetropAccess_YKR_grid Shapefile where from_id in Matrix file corresponds to YKR_ID in the Shapefile. The tool saves the result in the output-folder that user has defined. You should name the files in a way that it is possible to identify the ID from the name (e.g. 5797076).

##### Join the two tables into one geodataframe based on YKR_ID and from_id


In [65]:


#make in into a geodataframe
#tt_x = gpd.GeoDataFrame(tt_, crs=CRS, geometry ='geometry')

#check the output
#tt_x.head()

##### Save the geodataframe into shapefile which has a unique number (the YKR_ID)

In [20]:
tt_x.to_file(filename = YKR_ID)